In [7]:
import numpy as np
import torch
from gen import generate_vector
import json

In [8]:
dim = 8
save_path = "test_vector.npy"

In [9]:
input = generate_vector(cnt=3, dim=dim, mx=10, save_path=save_path)
print(input)

[[-0.71094584 -4.31944206 -9.13263801  9.2987121  -4.46483544  3.63263492
  -6.56336777 -6.50199216]
 [ 5.43638086  0.65285887  9.9046036  -9.77260702 -0.39844674 -0.54828253
  -9.37628447  7.06817142]
 [ 5.9102891  -6.95446651  1.36924797 -4.56595341 -1.366658   -2.27363413
  -4.79990715 -4.19061287]]


In [10]:
input_vector = np.load(save_path)
print(input_vector)

[[-0.71094584 -4.31944206 -9.13263801  9.2987121  -4.46483544  3.63263492
  -6.56336777 -6.50199216]
 [ 5.43638086  0.65285887  9.9046036  -9.77260702 -0.39844674 -0.54828253
  -9.37628447  7.06817142]
 [ 5.9102891  -6.95446651  1.36924797 -4.56595341 -1.366658   -2.27363413
  -4.79990715 -4.19061287]]


In [13]:
from pymilvus import MilvusClient, DataType, connections, db, Role, Collection, FieldSchema, CollectionSchema

connections.connect(host="127.0.0.1", port=19530)
if "test" not in db.list_database():
    db.create_database("test")
db.using_database("test")
client = MilvusClient(uri="http://localhost:19530", db_name="test")


In [14]:
id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="primary id")
vector_field = FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=dim, description="vector")

fields = [id_field, vector_field]
schema = CollectionSchema(fields=fields)

In [15]:
if "test" in client.list_collections():
    client.drop_collection("test")
collection = Collection(name="test", schema=schema)

In [16]:
index_params = client.prepare_index_params()
index_params = {
    "metric_type": "L2",
    "index_type": "GPU_IVF_PQ",
    "params": {
        "nlist": 3, "m":2
    }
}
collection.drop_index()
collection.create_index(
    field_name="vector",
    index_params=index_params
)

Status(code=0, message=)

In [17]:
index_name = client.list_indexes(
    collection_name="test"
)[0]

print(index_name)

_default_idx_101


In [18]:
print(json.dumps(collection.describe(), indent=2))
print(json.dumps(client.describe_index(collection_name="test",index_name=index_name), indent=2))

{
  "collection_name": "test",
  "auto_id": false,
  "num_shards": 1,
  "description": "",
  "fields": [
    {
      "field_id": 100,
      "name": "id",
      "description": "primary id",
      "type": 5,
      "params": {},
      "is_primary": true
    },
    {
      "field_id": 101,
      "name": "vector",
      "description": "vector",
      "type": 101,
      "params": {
        "dim": 8
      }
    }
  ],
  "aliases": [],
  "collection_id": 449369633535557748,
  "consistency_level": 2,
  "properties": {},
  "num_partitions": 1,
  "enable_dynamic_field": false
}
{
  "metric_type": "L2",
  "index_type": "GPU_IVF_PQ",
  "params": {
    "nlist": 3,
    "m": 2
  },
  "field_name": "vector",
  "index_name": "_default_idx_101"
}


In [19]:
n = 10
input = generate_vector(cnt=10, dim=dim, mx=10)
data = create_entries(input)
# print(data)

In [20]:
res = client.upsert(
    collection_name="test",
    data=data
)
print(res)

{'upsert_count': 10}


In [21]:
delete_ids = [2, 5, 8]

def create_delete_filter(delete_ids):
    return f"id in [{','.join([str(id) for id in delete_ids])}]"

res = client.delete(
    collection_name="test",
    filter=create_delete_filter(delete_ids)
)
print(res)


[2, 5, 8]


In [22]:
insert_ids = [2, 5, 8]
insert_vecs = generate_vector(cnt=3, dim=dim, mx=5)
insert_data = create_entries(insert_vecs, insert_ids)
client.insert(collection_name="test", data=insert_data)


{'insert_count': 3, 'ids': [2, 5, 8]}

In [23]:
upsert_ids = [2, 5, 8, 11]
upsert_vecs = generate_vector(cnt=4, dim=dim, mx=5)
upsert_data = create_entries(upsert_vecs, upsert_ids)
client.upsert(collection_name="test", data=upsert_data)

{'upsert_count': 4}

In [24]:
client.load_collection(collection_name="test")

In [25]:
res = client.search(
    collection_name='test',
    data=[[1,2,3,4,5,6,7,8]],
    limit=2,
    output_fields=['id'],
    filter='id > 1 and id < 7',
)

print(json.dumps(res, indent=4))

[
    [
        {
            "id": 3,
            "distance": 223.89996337890625,
            "entity": {
                "id": 3
            }
        },
        {
            "id": 2,
            "distance": 263.5295104980469,
            "entity": {
                "id": 2
            }
        }
    ]
]


In [58]:
import database_manager
database_manager.create_vector_index(client=client, collection_name="test",index_name="attempt",field_name="vector")
database_manager.create_scalar_index(client=client, collection_name="test")
print(client.list_indexes(collection_name="test"))

['scalar_index', 'attempt']


In [52]:
import importlib
importlib.reload(database_manager)

<module 'database_manage' from 'd:\\GPST\\mywork\\trymilvus\\database_manage.py'>